# Tier 0 — Baseline Training + Spectral Introspection (PACS, ResNet-18)

In [ ]:
# ===== Colab / Drive setup =====
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/SoMA_PACS"
import os
os.makedirs(PROJECT_ROOT, exist_ok=True)


Mounted at /content/drive


In [ ]:
!pip -q install datasets huggingface_hub pillow


In [ ]:
from datasets import load_dataset
import os
from PIL import Image
from tqdm import tqdm


In [ ]:
import shutil, os

PACS_ROOT = "/content/drive/MyDrive/NEW_PACS/PACS"
assert "PACS" in PACS_ROOT
if not os.path.exists(PACS_ROOT):
    os.makedirs(PACS_ROOT, exist_ok=True)
    print("Created PACS folder.")
else:
    print("PACS folder exists. Skipping deletion.")



PACS folder exists. Skipping deletion.


In [ ]:
# from datasets import load_dataset
# from tqdm.auto import tqdm
# import os

# ds = load_dataset("flwrlabs/pacs")

# DOMAIN_MAP = {
#     "photo": "photo",
#     "art_painting": "art_painting",
#     "cartoon": "cartoon",
#     "sketch": "sketch",
# }

# def safe(s):
#     return str(s).strip().lower().replace(" ", "_")

# # Per-domain counters to ensure unique filenames
# counters = {d: { } for d in DOMAIN_MAP.values()}

# for ex in tqdm(ds["train"], desc="Exporting PACS"):
#     img = ex["image"]

#     domain = ex["domain"]
#     label  = ex["label"]

#     if not isinstance(domain, str):
#         domain = ds["train"].features["domain"].int2str(domain)
#     if not isinstance(label, str):
#         label = ds["train"].features["label"].int2str(label)

#     domain = DOMAIN_MAP[safe(domain)]
#     label  = safe(label)

#     # initialize counter
#     counters[domain].setdefault(label, 0)
#     idx = counters[domain][label]
#     counters[domain][label] += 1

#     out_dir = os.path.join(PACS_ROOT, domain, label)
#     os.makedirs(out_dir, exist_ok=True)

#     out_path = os.path.join(out_dir, f"{label}_{idx:05d}.jpg")
#     img.save(out_path, quality=95)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/191M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9991 [00:00<?, ? examples/s]

Exporting PACS:   0%|          | 0/9991 [00:00<?, ?it/s]

In [ ]:
from torchvision import datasets
import os

for d in ["sketch","photo","art_painting","cartoon"]:
    ds = datasets.ImageFolder(os.path.join(PACS_ROOT, d))
    print(d, ds.class_to_idx)
    assert len(ds.class_to_idx) == 7, f"{d} does not have 7 classes!"



sketch {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
photo {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
art_painting {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
cartoon {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}


In [ ]:
from torchvision import datasets
import os

def get_class_to_idx(domain):
    ds = datasets.ImageFolder(root=os.path.join(PACS_ROOT, domain))
    return ds.class_to_idx

mappings = {d: get_class_to_idx(d) for d in ["photo","art_painting","cartoon","sketch"]}
for d, m in mappings.items():
    print(d, m)

# Compare
base = mappings["photo"]
for d in mappings:
    print(d, "matches photo:", mappings[d] == base)


photo {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
art_painting {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
cartoon {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
sketch {'dog': 0, 'elephant': 1, 'giraffe': 2, 'guitar': 3, 'horse': 4, 'house': 5, 'person': 6}
photo matches photo: True
art_painting matches photo: True
cartoon matches photo: True
sketch matches photo: True


In [ ]:
# ===== Reproducibility =====
import random, numpy as np, torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# ===== Imports =====
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm


In [ ]:
# ===== PACS paths (edit if needed) =====
# PACS_ROOT = "/content/drive/MyDrive/datasets/PACS"

SOURCE_DOMAINS = ["photo", "art_painting", "cartoon"]
TARGET_DOMAIN = "sketch"

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_WORKERS = 2


In [ ]:
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

test_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])


In [ ]:
def make_loader(domain, tf, shuffle):
    ds = datasets.ImageFolder(
        root=os.path.join(PACS_ROOT, domain),
        transform=tf
    )
    return DataLoader(ds, batch_size=BATCH_SIZE,
                      shuffle=shuffle, num_workers=NUM_WORKERS)

train_loaders = [make_loader(d, train_tf, True) for d in SOURCE_DOMAINS]
test_loader = make_loader(TARGET_DOMAIN, test_tf, False)


In [ ]:
class ResNet18_FeatureHook(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.resnet18(weights="IMAGENET1K_V1")
        self.backbone.fc = nn.Linear(512, num_classes)

        self._features = {}

        def make_hook(name):
            def hook(module, inp, out):
                self._features[name] = out
            return hook

        # IMPORTANT: include layer1 now
        for lname in ["layer1", "layer2", "layer3", "layer4"]:
            layer = getattr(self.backbone, lname)
            for i, block in enumerate(layer):
                block.bn2.register_forward_hook(
                    make_hook(f"{lname}.{i}.bn2")
                )

    def forward(self, x):
        self._features = {}
        return self.backbone(x)



# Training loop (Tier-0 backbone)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet18_FeatureHook(num_classes=7).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()


In [ ]:
def train_epoch(loaders):
    model.train()
    total_loss, total_correct, total = 0, 0, 0

    for loader in loaders:
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * x.size(0)
            total_correct += (logits.argmax(1) == y).sum().item()
            total += x.size(0)

    return total_loss / total, total_correct / total


In [ ]:
@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total_correct, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        total_correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)
    return total_correct / total


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
EPOCHS = 15
for ep in range(EPOCHS):
    tr_loss, tr_acc = train_epoch(train_loaders)
    te_acc = eval_epoch(test_loader)

    print(f"[{ep:02d}] loss={tr_loss:.3f} | src_acc={tr_acc:.3f} | tgt_acc={te_acc:.3f}")


[00] loss=0.386 | src_acc=0.874 | tgt_acc=0.661
[01] loss=0.213 | src_acc=0.929 | tgt_acc=0.632
[02] loss=0.144 | src_acc=0.953 | tgt_acc=0.680
[03] loss=0.123 | src_acc=0.958 | tgt_acc=0.652
[04] loss=0.085 | src_acc=0.972 | tgt_acc=0.725
[05] loss=0.058 | src_acc=0.981 | tgt_acc=0.718
[06] loss=0.105 | src_acc=0.967 | tgt_acc=0.733
[07] loss=0.063 | src_acc=0.982 | tgt_acc=0.708
[08] loss=0.061 | src_acc=0.981 | tgt_acc=0.688
[09] loss=0.074 | src_acc=0.976 | tgt_acc=0.754
[10] loss=0.031 | src_acc=0.990 | tgt_acc=0.722
[11] loss=0.028 | src_acc=0.991 | tgt_acc=0.732
[12] loss=0.063 | src_acc=0.979 | tgt_acc=0.699
[13] loss=0.044 | src_acc=0.986 | tgt_acc=0.708
[14] loss=0.038 | src_acc=0.987 | tgt_acc=0.736


Save trained backbone

In [ ]:
BACKBONE_PATH = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"

torch.save(model.state_dict(), BACKBONE_PATH)
print("Saved PACS-trained backbone.")


Saved PACS-trained backbone.


# TIER 0 — SPECTRAL INTROSPECTION

In [ ]:
import numpy.linalg as LA

def conv_svd(conv):
    # conv.weight: [C_out, C_in, k, k]
    W = conv.weight.detach().cpu().numpy()
    Cout = W.shape[0]
    Wmat = W.reshape(Cout, -1)
    U, S, Vt = LA.svd(Wmat, full_matrices=False)
    return U, S


In [ ]:
spectra = {}

for lname in ["layer2", "layer3", "layer4"]:
    for i, block in enumerate(getattr(model.backbone, lname)):
        conv = block.conv2
        U, S = conv_svd(conv)
        spectra[f"{lname}.{i}"] = S


In [ ]:
import pickle

with open(f"{PROJECT_ROOT}/tier0_spectra.pkl", "wb") as f:
    pickle.dump(spectra, f)


# Tier-0 Feature Statistics (Energy by Spectral Rank)

In [ ]:
@torch.no_grad()
def collect_feature_energy(loader):
    model.eval()
    energy = {}

    for x, _ in loader:
        x = x.to(device)
        _ = model(x)

        for k, feat in model._features.items():
            # GAP → channel vector
            h = feat.mean(dim=[2,3])  # [B, C]
            e = (h**2).mean(0).cpu().numpy()
            energy.setdefault(k, []).append(e)

    for k in energy:
        energy[k] = np.mean(energy[k], axis=0)

    return energy


In [ ]:
src_energy = collect_feature_energy(train_loaders[0])
tgt_energy = collect_feature_energy(test_loader)

with open(f"{PROJECT_ROOT}/tier0_feature_energy.pkl", "wb") as f:
    pickle.dump({"src": src_energy, "tgt": tgt_energy}, f)


Saved to Drive

- tier0_spectra.pkl → singular values per conv layer

- tier0_feature_energy.pkl → channel-wise activation energy (src vs tgt)

- trained baseline checkpoint (implicitly in model state)



Scientific baseline

- no spectral intervention

- no leakage

- fixed feature definition

- BN explicitly visible

# Tier 1A — Do Minor Singular Subspaces Encode More Domain Information?

Is the minor singular subspace more domain-specific, while the major subspace is more class-semantic?

In [ ]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy.linalg as LA

In [ ]:
with open(f"{PROJECT_ROOT}/tier0_spectra.pkl", "rb") as f:
    spectra = pickle.load(f)


In [ ]:
def get_conv_svd(conv):
    """
    Computes SVD of a conv layer in output-channel space.

    conv.weight: [C_out, C_in, k, k]
    Returns:
        U: [C_out, r]
        S: [r]
    """
    W = conv.weight.detach().cpu().numpy()
    C_out = W.shape[0]
    W_mat = W.reshape(C_out, -1)
    U, S, _ = LA.svd(W_mat, full_matrices=False)
    return U, S



Build Subspace Bases (Major / Minor / Random)

In [ ]:
def get_subspace(U, kind, r, seed=0):
    """
    Returns an orthonormal basis of shape [C_out, r]
    """
    C = U.shape[0]
    rng = np.random.default_rng(seed)

    if kind == "major":
        return U[:, :r]

    if kind == "minor":
        return U[:, -r:]

    if kind == "random":
        Q, _ = np.linalg.qr(rng.standard_normal((C, r)))
        return Q

    raise ValueError(f"Unknown subspace kind: {kind}")



In [ ]:
@torch.no_grad()
def extract_gap_features(domains, layer_name):
    DOMAIN_TO_ID = {
        "photo": 0,
        "art_painting": 1,
        "cartoon": 2,
        "sketch": 3,
    }

    X, y_class, y_domain = [], [], []
    model.eval()

    for domain in domains:
        loader = make_loader(domain, test_tf, shuffle=False)
        d_id = DOMAIN_TO_ID[domain]

        for x, y in loader:
            x = x.to(device)
            _ = model(x)

            feat = model._features[f"{layer_name}.bn2"]
            h = feat.mean(dim=[2, 3]).cpu().numpy()  # GAP

            X.append(h)
            y_class.append(y.numpy())
            y_domain.append(np.full(h.shape[0], d_id))

    return (
        np.concatenate(X),
        np.concatenate(y_class),
        np.concatenate(y_domain),
    )



In [ ]:
class ProjectedDataset(Dataset):
    def __init__(self, X, y, Usub):
        Z = X @ Usub
        self.X = torch.tensor(Z, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]



In [ ]:
class LinearProbe(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.fc = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        return self.fc(x)


def train_probe(dataset, num_classes, seed=0, epochs=50, train_frac=0.7):
    torch.manual_seed(seed)

    n = len(dataset)
    n_train = int(train_frac * n)
    n_val = n - n_train

    train_ds, val_ds = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

    probe = LinearProbe(dataset.X.shape[1], num_classes).to(device)
    opt = torch.optim.Adam(probe.parameters(), lr=1e-2)
    loss_fn = nn.CrossEntropyLoss()

    for _ in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss_fn(probe(x), y).backward()
            opt.step()

    correct, total = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = probe(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    return correct / total


# Tier-1.1: Domain probe (core SoMA assumption)

In [ ]:
# -------- Tier 1.1: Domain Probing --------

layers = {
    "layer2.1": model.backbone.layer2[1].conv2,
    "layer3.1": model.backbone.layer3[1].conv2,
    "layer4.1": model.backbone.layer4[1].conv2,
}

domains = ["photo", "art_painting", "cartoon", "sketch"]
ranks = [4, 8, 16]
seeds = [0, 1, 2]

for layer_name, conv in layers.items():
    print(f"\n=== Domain Probe @ {layer_name} ===")

    U, S = get_conv_svd(conv)
    X, _, y_domain = extract_gap_features(domains, model, layer_name)

    for r in ranks:
        print(f"\nRank {r}")
        for kind in ["major", "minor", "random"]:
            accs = []
            for seed in seeds:
                Usub = get_subspace(U, kind, r, seed)
                ds = ProjectedDataset(X, y_domain, Usub)
                accs.append(train_probe(ds, num_classes=4, seed=seed))

            print(f"{kind:>6}: {np.mean(accs):.3f} ± {np.std(accs):.3f}")



=== Domain Probe @ layer2.1 ===

Rank 4
 major: 0.678 ± 0.007
 minor: 0.672 ± 0.003
random: 0.624 ± 0.064

Rank 8
 major: 0.752 ± 0.003
 minor: 0.750 ± 0.003
random: 0.792 ± 0.023

Rank 16
 major: 0.842 ± 0.003
 minor: 0.823 ± 0.003
random: 0.826 ± 0.012

=== Domain Probe @ layer3.1 ===

Rank 4
 major: 0.704 ± 0.004
 minor: 0.547 ± 0.013
random: 0.659 ± 0.040

Rank 8
 major: 0.775 ± 0.007
 minor: 0.669 ± 0.010
random: 0.714 ± 0.053

Rank 16
 major: 0.800 ± 0.003
 minor: 0.731 ± 0.007
random: 0.822 ± 0.017

=== Domain Probe @ layer4.1 ===

Rank 4
 major: 0.535 ± 0.016
 minor: 0.478 ± 0.004
random: 0.583 ± 0.022

Rank 8
 major: 0.654 ± 0.001
 minor: 0.528 ± 0.012
random: 0.622 ± 0.012

Rank 16
 major: 0.730 ± 0.005
 minor: 0.603 ± 0.010
random: 0.708 ± 0.012


# Tier-1.2: Class probe (complementary assumption)

In [ ]:
# -------- Tier 1.2: Class Probing --------

source_domains = ["photo", "art_painting", "cartoon"]

for layer_name, conv in layers.items():
    print(f"\n=== Class Probe @ {layer_name} ===")

    U, S = get_conv_svd(conv)
    X, y_class, _ = extract_gap_features(source_domains, model, layer_name)

    for r in ranks:
        print(f"\nRank {r}")
        for kind in ["major", "minor", "random"]:
            accs = []
            for seed in seeds:
                Usub = get_subspace(U, kind, r, seed)
                ds = ProjectedDataset(X, y_class, Usub)
                accs.append(train_probe(ds, num_classes=7, seed=seed))

            print(f"{kind:>6}: {np.mean(accs):.3f} ± {np.std(accs):.3f}")



=== Class Probe @ layer2.1 ===

Rank 4
 major: 0.245 ± 0.006
 minor: 0.299 ± 0.002
random: 0.288 ± 0.016

Rank 8
 major: 0.336 ± 0.010
 minor: 0.331 ± 0.004
random: 0.361 ± 0.010

Rank 16
 major: 0.402 ± 0.014
 minor: 0.374 ± 0.005
random: 0.422 ± 0.012

=== Class Probe @ layer3.1 ===

Rank 4
 major: 0.342 ± 0.012
 minor: 0.285 ± 0.004
random: 0.338 ± 0.012

Rank 8
 major: 0.470 ± 0.002
 minor: 0.332 ± 0.010
random: 0.416 ± 0.018

Rank 16
 major: 0.624 ± 0.005
 minor: 0.431 ± 0.005
random: 0.528 ± 0.013

=== Class Probe @ layer4.1 ===

Rank 4
 major: 0.938 ± 0.004
 minor: 0.471 ± 0.005
random: 0.846 ± 0.018

Rank 8
 major: 0.970 ± 0.004
 minor: 0.704 ± 0.011
random: 0.936 ± 0.010

Rank 16
 major: 0.975 ± 0.001
 minor: 0.821 ± 0.004
random: 0.962 ± 0.007


Domain & Class Probe for all layers

In [ ]:
BACKBONE_PATH = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"

model = ResNet18_FeatureHook(num_classes=7).to(device)

state = torch.load(BACKBONE_PATH, map_location=device)
model.load_state_dict(state)

model.eval()


ResNet18_FeatureHook(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=T

Testing that the right model was loaded

In [ ]:
te_acc = eval_epoch(test_loader)
print("Target accuracy:", te_acc)


Target accuracy: 0.7358106388393993


In [ ]:
# Enumerate all conv2 layers in ResNet-18
conv_blocks = {}

for lname in ["layer1", "layer2", "layer3", "layer4"]:
    layer = getattr(model.backbone, lname)
    for i, block in enumerate(layer):
        key = f"{lname}.{i}"
        conv_blocks[key] = block.conv2


In [ ]:
domains_all = ["photo", "art_painting", "cartoon", "sketch"]
domains_source = ["photo", "art_painting", "cartoon"]

ranks = [4, 8, 16]
seeds = [0, 1, 2]


In [ ]:
print("===== DOMAIN PROBES (ALL CONV BLOCKS) =====")

for layer_name, conv in conv_blocks.items():
    print(f"\n=== Domain probe @ {layer_name} ===")

    # SVD of this conv layer
    U, S = get_conv_svd(conv)

    # Extract GAP features
    X, _, y_domain = extract_gap_features(domains_all, layer_name)

    for r in ranks:
        print(f"Rank {r}")
        for kind in ["major", "minor", "random"]:
            accs = []
            for seed in seeds:
                Usub = get_subspace(U, kind, r, seed)
                ds = ProjectedDataset(X, y_domain, Usub)
                accs.append(train_probe(ds, num_classes=4, seed=seed))

            print(f"{kind:>6}: {np.mean(accs):.3f} ± {np.std(accs):.3f}")


===== DOMAIN PROBES (ALL CONV BLOCKS) =====

=== Domain probe @ layer1.0 ===
Rank 4
 major: 0.749 ± 0.004
 minor: 0.585 ± 0.016
random: 0.656 ± 0.031
Rank 8
 major: 0.772 ± 0.004
 minor: 0.715 ± 0.017
random: 0.730 ± 0.026
Rank 16
 major: 0.843 ± 0.004
 minor: 0.780 ± 0.003
random: 0.828 ± 0.014

=== Domain probe @ layer1.1 ===
Rank 4
 major: 0.601 ± 0.010
 minor: 0.617 ± 0.013
random: 0.626 ± 0.076
Rank 8
 major: 0.700 ± 0.009
 minor: 0.706 ± 0.012
random: 0.705 ± 0.030
Rank 16
 major: 0.823 ± 0.002
 minor: 0.797 ± 0.004
random: 0.805 ± 0.029

=== Domain probe @ layer2.0 ===
Rank 4
 major: 0.607 ± 0.008
 minor: 0.733 ± 0.015
random: 0.661 ± 0.054
Rank 8
 major: 0.739 ± 0.008
 minor: 0.775 ± 0.010
random: 0.791 ± 0.018
Rank 16
 major: 0.835 ± 0.006
 minor: 0.836 ± 0.007
random: 0.836 ± 0.008

=== Domain probe @ layer2.1 ===
Rank 4
 major: 0.678 ± 0.007
 minor: 0.672 ± 0.003
random: 0.624 ± 0.064
Rank 8
 major: 0.752 ± 0.003
 minor: 0.750 ± 0.003
random: 0.792 ± 0.023
Rank 16
 major: 0.

In [ ]:
print("\n===== CLASS PROBES (ALL CONV BLOCKS) =====")

for layer_name, conv in conv_blocks.items():
    print(f"\n=== Class probe @ {layer_name} ===")

    # SVD of this conv layer
    U, S = get_conv_svd(conv)

    # Extract GAP features (source domains only)
    X, y_class, _ = extract_gap_features(domains_source, layer_name)

    for r in ranks:
        print(f"Rank {r}")
        for kind in ["major", "minor", "random"]:
            accs = []
            for seed in seeds:
                Usub = get_subspace(U, kind, r, seed)
                ds = ProjectedDataset(X, y_class, Usub)
                accs.append(train_probe(ds, num_classes=7, seed=seed))

            print(f"{kind:>6}: {np.mean(accs):.3f} ± {np.std(accs):.3f}")



===== CLASS PROBES (ALL CONV BLOCKS) =====

=== Class probe @ layer1.0 ===
Rank 4
 major: 0.299 ± 0.008
 minor: 0.265 ± 0.007
random: 0.278 ± 0.011
Rank 8
 major: 0.332 ± 0.002
 minor: 0.296 ± 0.003
random: 0.326 ± 0.017
Rank 16
 major: 0.394 ± 0.004
 minor: 0.348 ± 0.013
random: 0.366 ± 0.004

=== Class probe @ layer1.1 ===
Rank 4
 major: 0.279 ± 0.008
 minor: 0.236 ± 0.005
random: 0.274 ± 0.018
Rank 8
 major: 0.331 ± 0.006
 minor: 0.292 ± 0.005
random: 0.301 ± 0.020
Rank 16
 major: 0.397 ± 0.005
 minor: 0.363 ± 0.007
random: 0.373 ± 0.005

=== Class probe @ layer2.0 ===
Rank 4
 major: 0.262 ± 0.012
 minor: 0.234 ± 0.008
random: 0.287 ± 0.017
Rank 8
 major: 0.378 ± 0.009
 minor: 0.321 ± 0.011
random: 0.358 ± 0.019
Rank 16
 major: 0.439 ± 0.007
 minor: 0.366 ± 0.002
random: 0.409 ± 0.005

=== Class probe @ layer2.1 ===
Rank 4
 major: 0.245 ± 0.006
 minor: 0.299 ± 0.002
random: 0.288 ± 0.016
Rank 8
 major: 0.336 ± 0.010
 minor: 0.331 ± 0.004
random: 0.361 ± 0.010
Rank 16
 major: 0.402 

# Tier-2: ΔW-Based Domain-Sensitive Subspace Discovery

In [ ]:
import copy
import numpy as np
import torch
import torch.nn as nn
import numpy.linalg as LA


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


Load frozen Tier-0 backbone

In [ ]:
BACKBONE_PATH = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"

def load_base_model():
    model = ResNet18_FeatureHook(num_classes=7).to(device)
    state = torch.load(BACKBONE_PATH, map_location=device)
    model.load_state_dict(state)
    return model


Freeze BatchNorm running statistics: This prevents BN from silently encoding domain identity.

In [ ]:
def freeze_bn_stats(model):
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            m.eval()                  # freeze running mean/var
            m.weight.requires_grad = True
            m.bias.requires_grad = True


Class-balanced domain loader - This ensures ΔW reflects domain, not class imbalance.



In [ ]:
from collections import defaultdict

def make_balanced_loader(domain, tf, batch_size):
    ds = datasets.ImageFolder(
        root=os.path.join(PACS_ROOT, domain),
        transform=tf
    )

    # group indices by class
    class_to_indices = defaultdict(list)
    for idx, (_, y) in enumerate(ds.samples):
        class_to_indices[y].append(idx)

    # sample equal counts per class
    min_count = min(len(v) for v in class_to_indices.values())
    balanced_indices = []
    for v in class_to_indices.values():
        balanced_indices.extend(v[:min_count])

    sampler = torch.utils.data.SubsetRandomSampler(balanced_indices)

    return DataLoader(
        ds,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=NUM_WORKERS
    )


Train single-domain model (light adaptation)

In [ ]:
def train_single_domain(domain, epochs=2, lr=1e-4):
    model = load_base_model()
    freeze_bn_stats(model)

    loader = make_balanced_loader(domain, train_tf, BATCH_SIZE)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for _ in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()

    return model

# Note:

# Few epochs

# Small learning rate

# No over-fitting

# This captures early domain pressure.


Extract ΔW for a convolutional layer - ΔW lives in exactly the same space as Tier-1 SVD.

In [ ]:
def extract_delta_W(base_model, adapted_model, conv_path):
    """
    conv_path example: ("layer3", 1, "conv2")
    """
    base_conv = getattr(
        getattr(base_model.backbone, conv_path[0])[conv_path[1]],
        conv_path[2]
    )

    adapt_conv = getattr(
        getattr(adapted_model.backbone, conv_path[0])[conv_path[1]],
        conv_path[2]
    )

    W0 = base_conv.weight.detach().cpu().numpy()
    W1 = adapt_conv.weight.detach().cpu().numpy()

    dW = W1 - W0
    C_out = dW.shape[0]
    return dW.reshape(C_out, -1)


Collect ΔW across domains

In [ ]:
# Layer1 is intentionally excluded because it mostly captures low-level visual primitives (edges, textures), where ΔW is dominated by generic statistics rather than domain-specific adaptation.

conv_layers = {
    "layer2.1": ("layer2", 1, "conv2"),
    "layer3.1": ("layer3", 1, "conv2"),
    "layer4.1": ("layer4", 1, "conv2"),
}

base_model = load_base_model()
deltaWs = {k: [] for k in conv_layers}

for domain in SOURCE_DOMAINS:
    print(f"Adapting to domain: {domain}")
    adapted = train_single_domain(domain)

    for lname, path in conv_layers.items():
        dW = extract_delta_W(base_model, adapted, path)
        deltaWs[lname].append(dW)


Adapting to domain: photo
Adapting to domain: art_painting
Adapting to domain: cartoon


In [ ]:
import numpy as np
import os

SAVE_PATH = f"{PROJECT_ROOT}/tier2_deltaWs_source.npz"

np.savez(
    SAVE_PATH,
    **{
        f"{lname}_{i}": dW
        for lname, dW_list in deltaWs.items()
        for i, dW in enumerate(dW_list)
    }
)

print("Saved raw ΔW matrices to:", SAVE_PATH)


Saved raw ΔW matrices to: /content/drive/MyDrive/SoMA_PACS/tier2_deltaWs_source.npz


In [ ]:
def load_deltaWs(path, conv_layers, num_domains=3):
    data = np.load(path)
    deltaWs = {lname: [] for lname in conv_layers}

    for lname in conv_layers:
        for i in range(num_domains):
            deltaWs[lname].append(data[f"{lname}_{i}"])

    return deltaWs

deltaWs = load_deltaWs(
    f"{PROJECT_ROOT}/tier2_deltaWs_source.npz",
    conv_layers,
    num_domains=len(SOURCE_DOMAINS)
)

print("Loaded ΔW matrices from disk.")


Loaded ΔW matrices from disk.


In [ ]:
np.savez(
    f"{PROJECT_ROOT}/tier2_deltaW_subspaces.npz",
    **{
        f"{lname}_U": U
        for lname, (U, S) in delta_subspaces.items()
    }
)

print("Saved ΔW subspaces.")


Saved ΔW subspaces.


In [ ]:
def load_deltaW_subspaces(path, conv_layers):
    data = np.load(path)
    return {
        lname: (data[f"{lname}_U"], None)
        for lname in conv_layers
    }

delta_subspaces = load_deltaW_subspaces(
    f"{PROJECT_ROOT}/tier2_deltaW_subspaces.npz",
    conv_layers
)

print("Loaded ΔW subspaces.")


Loaded ΔW subspaces.


# Tier-2.2 — Empirical Domain-Sensitive Subspace (Channel Space)

Build channel–channel covariance from ΔW

In [ ]:
def build_deltaW_channel_subspace(dW_list):
    """
    dW_list: list of ΔW matrices for one layer
             each ΔW has shape [C_out, D]

    Returns:
        U_domain: [C_out, C_out] eigenvectors (descending order)
        S_domain: eigenvalues
    """
    C_out = dW_list[0].shape[0]
    C = np.zeros((C_out, C_out))

    for dW in dW_list:
        C += dW @ dW.T   # channel-channel covariance

    # Eigen-decomposition (symmetric PSD matrix)
    eigvals, eigvecs = np.linalg.eigh(C)

    # Sort descending
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    return eigvecs, eigvals


Construct ΔW domain subspaces per layer

In [ ]:
delta_subspaces = {}

for lname, dW_list in deltaWs.items():
    U_domain, S_domain = build_deltaW_channel_subspace(dW_list)
    delta_subspaces[lname] = (U_domain, S_domain)

print("Constructed empirical ΔW domain-sensitive subspaces.")


Constructed empirical ΔW domain-sensitive subspaces.


# E2.3 — ALIGNMENT WITH SoMA MINOR SUBSPACE

SoMA subspace constructor

In [ ]:
# def get_soma_subspace(conv, kind, r):
#     """
#     kind: 'minor', 'major', or 'random'
#     Returns: [C_out, r] orthonormal basis
#     """
#     W = conv.weight.detach().cpu().numpy()
#     C_out = W.shape[0]
#     Wmat = W.reshape(C_out, -1)

#     U, S, _ = np.linalg.svd(Wmat, full_matrices=False)

#     if kind == "minor":
#         return U[:, -r:]
#     if kind == "major":
#         return U[:, :r]
#     if kind == "random":
#         Q, _ = np.linalg.qr(np.random.randn(C_out, r))
#         return Q

#     raise ValueError

Alignment metric computation

In [ ]:
def compute_alignment_metrics(U, V):
    """
    U, V: [C_out, r] orthonormal basis matrices

    Returns:
        mean_angle_deg
        overlap (mean cos^2)
        projection_energy
    """
    # Cross-subspace matrix
    M = U.T @ V   # [r, r]

    # Singular values = cos(theta_i)
    sv = np.linalg.svd(M, compute_uv=False)
    sv = np.clip(sv, -1.0, 1.0)

    # Principal angles (degrees)
    angles = np.degrees(np.arccos(sv))
    mean_angle = angles.mean()

    # Overlap (mean squared cosine)
    overlap = np.mean(sv ** 2)

    # Projection energy (equivalent to overlap)
    proj_energy = np.linalg.norm(M, ord="fro") ** 2 / V.shape[1]

    return mean_angle, overlap, proj_energy


Alignment sweep (SoMA vs ΔW)

In [ ]:
# Build conv_blocks mapping (SoMA reference layers)

conv_layers = {
    "layer2.1": ("layer2", 1, "conv2"),
    "layer3.1": ("layer3", 1, "conv2"),
    "layer4.1": ("layer4", 1, "conv2"),
}
base_model = load_base_model()
conv_blocks = {
    "layer2.1": base_model.backbone.layer2[1].conv2,
    "layer3.1": base_model.backbone.layer3[1].conv2,
    "layer4.1": base_model.backbone.layer4[1].conv2,
}

print("Defined conv_blocks:", list(conv_blocks.keys()))


Defined conv_blocks: ['layer2.1', 'layer3.1', 'layer4.1']


In [ ]:
ranks = [2, 4, 8, 16]
n_random = 10

alignment_results = {}

for lname, (U_domain, S_domain) in delta_subspaces.items():
    print(f"\n=== Alignment @ {lname} ===")
    alignment_results[lname] = {}

    conv = conv_blocks[lname]

    for r in ranks:
        print(f"\nRank {r}")
        V = U_domain[:, :r]   # empirical ΔW subspace

        alignment_results[lname][r] = {}

        for kind in ["minor", "major", "random"]:
            angles, overlaps, energies = [], [], []

            for seed in range(n_random):
                np.random.seed(seed)
                U = get_soma_subspace(conv, kind, r)

                mean_angle, overlap, proj_energy = compute_alignment_metrics(U, V)

                angles.append(mean_angle)
                overlaps.append(overlap)
                energies.append(proj_energy)

            alignment_results[lname][r][kind] = {
                "mean_angle_deg": (np.mean(angles), np.std(angles)),
                "overlap": (np.mean(overlaps), np.std(overlaps)),
                "proj_energy": (np.mean(energies), np.std(energies)),
            }

            print(
                f"{kind:>6} | "
                f"angle={np.mean(angles):.2f}° ± {np.std(angles):.2f} | "
                f"overlap={np.mean(overlaps):.3f} ± {np.std(overlaps):.3f} | "
                f"energy={np.mean(energies):.3f} ± {np.std(energies):.3f}"
            )



=== Alignment @ layer2.1 ===

Rank 2
 minor | angle=88.70° ± 0.00 | overlap=0.001 ± 0.000 | energy=0.001 ± 0.000
 major | angle=71.25° ± 0.00 | overlap=0.123 ± 0.000 | energy=0.123 ± 0.000
random | angle=84.26° ± 2.56 | overlap=0.014 ± 0.011 | energy=0.014 ± 0.011

Rank 4
 minor | angle=84.59° ± 0.00 | overlap=0.012 ± 0.000 | energy=0.012 ± 0.000
 major | angle=75.95° ± 0.00 | overlap=0.085 ± 0.000 | energy=0.085 ± 0.000
random | angle=81.61° ± 1.29 | overlap=0.031 ± 0.009 | energy=0.031 ± 0.009

Rank 8
 minor | angle=82.35° ± 0.00 | overlap=0.024 ± 0.000 | energy=0.024 ± 0.000
 major | angle=70.81° ± 0.00 | overlap=0.146 ± 0.000 | energy=0.146 ± 0.000
random | angle=77.92° ± 1.28 | overlap=0.061 ± 0.012 | energy=0.061 ± 0.012

Rank 16
 minor | angle=78.92° ± 0.00 | overlap=0.051 ± 0.000 | energy=0.051 ± 0.000
 major | angle=61.52° ± 0.00 | overlap=0.271 ± 0.000 | energy=0.271 ± 0.000
random | angle=72.06° ± 0.96 | overlap=0.125 ± 0.012 | energy=0.125 ± 0.012

=== Alignment @ layer3.1

(Optional) Save results

In [ ]:
import pandas as pd

rows = []
for lname, layer_data in alignment_results.items():
    for r, r_data in layer_data.items():
        for kind, metrics in r_data.items():
            rows.append({
                "layer": lname,
                "rank": r,
                "subspace": kind,
                "mean_angle_deg": metrics["mean_angle_deg"][0],
                "angle_std": metrics["mean_angle_deg"][1],
                "overlap": metrics["overlap"][0],
                "overlap_std": metrics["overlap"][1],
                "proj_energy": metrics["proj_energy"][0],
                "energy_std": metrics["proj_energy"][1],
            })

df = pd.DataFrame(rows)
df.to_csv(f"{PROJECT_ROOT}/tier2_alignment_metrics.csv", index=False)

print("Saved Tier-2.3 alignment results.")

In [ ]:
print("Adapting baseline model to Sketch domain...")
adapted_sketch = train_single_domain("sketch")


Adapting baseline model to Sketch domain...


In [ ]:
deltaWs_sketch = {}

for lname, path in conv_layers.items():
    dW = extract_delta_W(base_model, adapted_sketch, path)
    deltaWs_sketch[lname] = dW
    print(f"{lname}: ΔW shape = {dW.shape}")


layer2.1: ΔW shape = (128, 1152)
layer3.1: ΔW shape = (256, 2304)
layer4.1: ΔW shape = (512, 4608)


Convert Sketch ΔW into a subspace

In [ ]:
def get_deltaW_subspace(dW, r):
    """
    Convert ΔW matrix into rank-r subspace via SVD.
    """
    U, S, _ = np.linalg.svd(dW, full_matrices=False)
    return U[:, :r]


Align Sketch ΔW subspace with SoMA subspaces

In [ ]:
ranks = [2, 4, 8, 16]
n_random = 10

print("\n=== Sketch ΔW vs SoMA Subspace Alignment ===")

sketch_alignment_results = {}

for lname in conv_layers:
    print(f"\n=== Layer {lname} ===")
    sketch_alignment_results[lname] = {}

    dW = deltaWs_sketch[lname]
    conv = conv_blocks[lname]

    for r in ranks:
        print(f"\nRank {r}")
        sketch_alignment_results[lname][r] = {}

        # Sketch ΔW subspace
        U_sketch = get_deltaW_subspace(dW, r)

        for kind in ["minor", "major", "random"]:
            angles, overlaps, energies = [], [], []

            for seed in range(n_random):
                np.random.seed(seed)

                # SoMA subspace
                U_soma = get_soma_subspace(conv, kind, r)

                mean_angle, overlap, proj_energy = compute_alignment_metrics(
                    U_soma, U_sketch
                )

                angles.append(mean_angle)
                overlaps.append(overlap)
                energies.append(proj_energy)

            sketch_alignment_results[lname][r][kind] = {
                "mean_angle_deg": (np.mean(angles), np.std(angles)),
                "overlap": (np.mean(overlaps), np.std(overlaps)),
                "proj_energy": (np.mean(energies), np.std(energies)),
            }

            print(
                f"{kind:>6} | "
                f"angle={np.mean(angles):.2f}° ± {np.std(angles):.2f} | "
                f"overlap={np.mean(overlaps):.3f} ± {np.std(overlaps):.3f} | "
                f"energy={np.mean(energies):.3f} ± {np.std(energies):.3f}"
            )



=== Sketch ΔW vs SoMA Subspace Alignment ===

=== Layer layer2.1 ===

Rank 2
 minor | angle=86.56° ± 0.00 | overlap=0.006 ± 0.000 | energy=0.006 ± 0.000
 major | angle=80.82° ± 0.00 | overlap=0.035 ± 0.000 | energy=0.035 ± 0.000
random | angle=85.68° ± 1.70 | overlap=0.008 ± 0.006 | energy=0.008 ± 0.006

Rank 4
 minor | angle=84.65° ± 0.00 | overlap=0.014 ± 0.000 | energy=0.014 ± 0.000
 major | angle=79.65° ± 0.00 | overlap=0.052 ± 0.000 | energy=0.052 ± 0.000
random | angle=81.75° ± 1.56 | overlap=0.030 ± 0.011 | energy=0.030 ± 0.011

Rank 8
 minor | angle=81.52° ± 0.00 | overlap=0.033 ± 0.000 | energy=0.033 ± 0.000
 major | angle=72.98° ± 0.00 | overlap=0.119 ± 0.000 | energy=0.119 ± 0.000
random | angle=77.64° ± 1.45 | overlap=0.065 ± 0.014 | energy=0.065 ± 0.014

Rank 16
 minor | angle=77.17° ± 0.00 | overlap=0.072 ± 0.000 | energy=0.072 ± 0.000
 major | angle=64.74° ± 0.00 | overlap=0.224 ± 0.000 | energy=0.224 ± 0.000
random | angle=72.19° ± 0.45 | overlap=0.123 ± 0.005 | energy

Check if Sketch's ΔW aligns with 3-domain ΔW subspace

In [ ]:
def project_matrix_onto_subspace(U, dW):
    """
    U  : [C_out, r] subspace basis (3-domain ΔW subspace)
    dW : [C_out, D] Sketch ΔW matrix

    Returns:
        projection_energy (fraction of ΔW energy captured)
        effective_angle_deg
    """
    # Project ΔW onto subspace
    P = U.T @ dW

    proj_energy = (np.linalg.norm(P, ord="fro") ** 2) / \
                  (np.linalg.norm(dW, ord="fro") ** 2)

    # Effective angle (for interpretability)
    eff_angle = np.degrees(
        np.arccos(np.sqrt(np.clip(proj_energy, 0.0, 1.0)))
    )

    return proj_energy, eff_angle


In [ ]:
ranks = [2, 4, 8, 16]

print("\n=== Sketch ΔW vs 3-Domain ΔW Subspace (Approach B) ===")

sketch_vs_deltaW_results = {}

for lname, (U_domain, S_domain) in delta_subspaces.items():
    print(f"\n=== Layer {lname} ===")
    sketch_vs_deltaW_results[lname] = {}

    dW_sketch = deltaWs_sketch[lname]
    C_out = dW_sketch.shape[0]

    for r in ranks:
        U_sub = U_domain[:, :r]

        proj_energy, eff_angle = project_matrix_onto_subspace(
            U_sub, dW_sketch
        )

        random_baseline = r / C_out

        sketch_vs_deltaW_results[lname][r] = {
            "projection_energy": proj_energy,
            "effective_angle_deg": eff_angle,
            "random_baseline": random_baseline
        }

        print(
            f"Rank {r:2d} | "
            f"proj={proj_energy:.3f} | "
            f"angle={eff_angle:.1f}° | "
            f"random≈{random_baseline:.3f}"
        )



=== Sketch ΔW vs 3-Domain ΔW Subspace (Approach B) ===

=== Layer layer2.1 ===
Rank  2 | proj=0.041 | angle=78.4° | random≈0.016
Rank  4 | proj=0.097 | angle=71.8° | random≈0.031
Rank  8 | proj=0.173 | angle=65.4° | random≈0.062
Rank 16 | proj=0.289 | angle=57.5° | random≈0.125

=== Layer layer3.1 ===
Rank  2 | proj=0.039 | angle=78.6° | random≈0.008
Rank  4 | proj=0.083 | angle=73.3° | random≈0.016
Rank  8 | proj=0.154 | angle=66.9° | random≈0.031
Rank 16 | proj=0.256 | angle=59.6° | random≈0.062

=== Layer layer4.1 ===
Rank  2 | proj=0.208 | angle=62.8° | random≈0.004
Rank  4 | proj=0.482 | angle=46.0° | random≈0.008
Rank  8 | proj=0.666 | angle=35.3° | random≈0.016
Rank 16 | proj=0.708 | angle=32.7° | random≈0.031


# Tier 3.1 — Subspace Swap Experiment

In [ ]:
import torch, numpy as np, random

def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(0)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Path to the baseline (source-trained) ResNet-18
base_model_path = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"

def load_baseline_model(path):
    model = ResNet18_FeatureHook(num_classes=7).to(device)
    state = torch.load(path, map_location=device)
    model.load_state_dict(state)
    return model


In [ ]:
def get_soma_subspace(conv, kind, r, seed=0):
    """
    Returns an orthonormal basis U [C_out, r]
    for the requested subspace of conv2 weights.
    """
    W = conv.weight.detach().cpu().numpy()
    C_out = W.shape[0]
    Wmat = W.reshape(C_out, -1)

    U, S, _ = np.linalg.svd(Wmat, full_matrices=False)

    if kind == "major":
        return U[:, :r]

    if kind == "minor":
        return U[:, -r:]

    if kind == "middle":
        start = (C_out // 2) - (r // 2)
        return U[:, start:start + r]

    if kind == "random":
        rng = np.random.default_rng(seed)
        Q, _ = np.linalg.qr(rng.standard_normal((C_out, r)))
        return Q

    raise ValueError(f"Unknown subspace kind: {kind}")


In [ ]:
# ===== Tier-3 Data Loaders =====

train_loaders = [
    make_balanced_loader("photo", train_tf, BATCH_SIZE),
    make_balanced_loader("art_painting", train_tf, BATCH_SIZE),
    make_balanced_loader("cartoon", train_tf, BATCH_SIZE),
]

test_loader = make_loader("sketch", test_tf, shuffle=False)

print("Tier-3 loaders ready.")


Tier-3 loaders ready.


Gradient projection hook (core mechanism)

In [ ]:
def register_gradient_projection(param, U_np):
    """
    param : torch.nn.Parameter
    U_np  : numpy array [C_out, r]
    """

    # Convert once, outside the hook
    U = torch.tensor(U_np, dtype=torch.float32, device=param.device)
    U.requires_grad_(False)

    def hook(grad):
        # grad: [C_out, C_in, k, k]
        g = grad.view(grad.shape[0], -1)     # [C_out, D]
        g_proj = U @ (U.T @ g)               # projection in torch
        return g_proj.view_as(grad)

    param.register_hook(hook)



Apply subspace constraint to selected layers

In [ ]:
def apply_subspace_constraints(model, conv_layers, subspaces):
    """
    subspaces: {layer_name: U (C_out x r)}
    """
    for lname, U in subspaces.items():
        layer, idx, conv_name = conv_layers[lname]
        conv = getattr(getattr(model.backbone, layer)[idx], conv_name)
        register_gradient_projection(conv.weight, U)


Training loop (shared across all variants)

In [ ]:
def train_with_constraints(
    model,
    train_loaders,
    epochs=3,
    lr=1e-4
):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    model.train()
    for ep in range(epochs):
        for loader in train_loaders:
            for x, y in loader:
                x, y = x.to(device), y.to(device)

                optimizer.zero_grad()
                loss = criterion(model(x), y)
                loss.backward()
                optimizer.step()



Evaluation (target domain)

In [ ]:
@torch.no_grad()
def eval_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return correct / total


In [ ]:
ranks = [4, 8, 16]
variants = ["minor", "major", "middle", "random"]

tier3_results = {}

for r in ranks:
    print(f"\n=== Rank {r} ===")
    tier3_results[r] = {}

    for variant in variants:
        print(f"\nVariant: {variant}")

        # 1. Fresh baseline
        model = load_baseline_model(base_model_path)

        # 2. Build subspaces
        subspaces = {}
        for lname, (layer, idx, conv_name) in conv_layers.items():
            conv = getattr(getattr(model.backbone, layer)[idx], conv_name)
            U = get_soma_subspace(conv, variant, r)
            subspaces[lname] = U

        # 3. Apply constraints
        apply_subspace_constraints(model, conv_layers, subspaces)

        # 4. Train
        train_with_constraints(model, train_loaders)

        # 5. Evaluate
        acc = eval_accuracy(model, test_loader)
        tier3_results[r][variant] = acc

        print(f"Target accuracy: {acc:.3f}")



=== Rank 4 ===

Variant: minor
Target accuracy: 0.711

Variant: major
Target accuracy: 0.744

Variant: middle
Target accuracy: 0.735

Variant: random
Target accuracy: 0.728

=== Rank 8 ===

Variant: minor
Target accuracy: 0.658

Variant: major
Target accuracy: 0.745

Variant: middle
Target accuracy: 0.717

Variant: random
Target accuracy: 0.728

=== Rank 16 ===

Variant: minor
Target accuracy: 0.729

Variant: major
Target accuracy: 0.737

Variant: middle
Target accuracy: 0.712

Variant: random
Target accuracy: 0.730


# Tier 3.4 — BatchNorm Confound Controls

In [ ]:
import os, copy, random
import numpy as np
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
# You must have PROJECT_ROOT already
base_model_path = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"
assert os.path.exists(base_model_path), f"Missing baseline weights at {base_model_path}"

def load_baseline_model():
    model = ResNet18_FeatureHook(num_classes=7).to(device)
    state = torch.load(base_model_path, map_location=device)
    model.load_state_dict(state)
    return model


In [ ]:
# Uses your existing make_balanced_loader, make_loader, train_tf, test_tf, BATCH_SIZE
train_loaders = [
    make_balanced_loader("photo", train_tf, BATCH_SIZE),
    make_balanced_loader("art_painting", train_tf, BATCH_SIZE),
    make_balanced_loader("cartoon", train_tf, BATCH_SIZE),
]
test_loader = make_loader("sketch", test_tf, shuffle=False)

print("Tier-3 data loaders ready.")


Tier-3 data loaders ready.


In [ ]:
def apply_subspace_constraints(model, conv_layers, kind, r, seed=0):
    """
    Apply gradient projection constraints to conv2 weights.

    model       : nn.Module
    conv_layers : dict {lname: (layer, idx, conv_name)}
    kind        : {"minor", "major", "middle", "random"}
    r           : rank of subspace
    seed        : random seed (used only for 'random')
    """
    for lname, (layer, idx, conv_name) in conv_layers.items():
        conv = getattr(getattr(model.backbone, layer)[idx], conv_name)

        # get subspace basis (numpy)
        U = get_soma_subspace(conv, kind, r, seed=seed)

        # register gradient projection hook
        register_gradient_projection(conv.weight, U)


In [ ]:
def set_bn_mode(model, mode: str):
    """
    mode in {"bn_default", "bn_frozen", "bn_affine_only"}
    """
    assert mode in {"bn_default", "bn_frozen", "bn_affine_only"}

    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            if mode == "bn_default":
                m.train()  # stats update
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)

            elif mode == "bn_frozen":
                m.eval()   # stats frozen
                m.weight.requires_grad_(False)
                m.bias.requires_grad_(False)

            elif mode == "bn_affine_only":
                m.eval()   # stats frozen
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)


def replace_bn_with_gn(module: nn.Module, num_groups: int = 32):
    """
    Recursively replace all BatchNorm2d layers with GroupNorm,
    ensuring the new layers are placed on the same device.
    """
    for name, child in module.named_children():
        if isinstance(child, nn.BatchNorm2d):
            num_channels = child.num_features

            # choose valid group count
            g = min(num_groups, num_channels)
            while num_channels % g != 0 and g > 1:
                g -= 1

            gn = nn.GroupNorm(
                num_groups=g,
                num_channels=num_channels,
                affine=True
            )

            # 🔑 MOVE TO SAME DEVICE AS ORIGINAL BN
            gn = gn.to(child.weight.device)

            setattr(module, name, gn)

        else:
            replace_bn_with_gn(child, num_groups=num_groups)



def apply_norm_config(model, norm_cfg: str):
    """
    norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}
    """
    assert norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}

    if norm_cfg == "groupnorm":
        replace_bn_with_gn(model)
        # GroupNorm has no running stats; keep train mode.
        model.train()
    else:
        set_bn_mode(model, norm_cfg)

    return model


In [ ]:
def get_soma_subspace(conv, kind, r, seed=0):
    """
    Returns U basis [C_out, r] as numpy array (orthonormal).
    """
    W = conv.weight.detach().cpu().numpy()
    C_out = W.shape[0]
    Wmat = W.reshape(C_out, -1)
    U, S, _ = np.linalg.svd(Wmat, full_matrices=False)

    if kind == "major":
        return U[:, :r]
    if kind == "minor":
        return U[:, -r:]
    if kind == "middle":
        start = (C_out // 2) - (r // 2)
        return U[:, start:start + r]
    if kind == "random":
        rng = np.random.default_rng(seed)
        Q, _ = np.linalg.qr(rng.standard_normal((C_out, r)))
        return Q

    raise ValueError(f"Unknown kind: {kind}")


In [ ]:
def register_gradient_projection(param, U_np):
    """
    Project gradient onto span(U).
    U_np is numpy [C_out, r].
    """
    U = torch.tensor(U_np, dtype=torch.float32, device=param.device)
    U.requires_grad_(False)

    def hook(grad):
        g = grad.view(grad.shape[0], -1)         # [C_out, D]
        g_proj = U @ (U.T @ g)                   # [C_out, D]
        return g_proj.view_as(grad)

    param.register_hook(hook)


def apply_subspace_constraints(model, conv_layers, kind, r, seed=0):
    """
    Apply gradient projection to conv2 weights for selected layers.
    conv_layers: dict {lname: (layer, idx, "conv2")}
    """
    for lname, (layer, idx, conv_name) in conv_layers.items():
        conv = getattr(getattr(model.backbone, layer)[idx], conv_name)
        U = get_soma_subspace(conv, kind, r, seed=seed)
        register_gradient_projection(conv.weight, U)


In [ ]:
@torch.no_grad()
def eval_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return correct / total


def train_with_logging(model, train_loaders, epochs=3, lr=1e-4):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    logs = {
        "epoch_loss": [],
        "grad_norm": [],
        "update_norm": [],
    }

    # snapshot params for update norm
    with torch.no_grad():
        init_params = {n: p.detach().clone() for n, p in model.named_parameters() if p.requires_grad}

    model.train()
    for ep in range(epochs):
        ep_loss_sum, ep_n = 0.0, 0
        grad_norm_sum, grad_n = 0.0, 0

        for loader in train_loaders:
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()
                logits = model(x)
                loss = criterion(logits, y)
                loss.backward()

                # grad norm (after projection, since hooks already applied)
                with torch.no_grad():
                    total_g = 0.0
                    for p in model.parameters():
                        if p.grad is not None:
                            total_g += p.grad.detach().float().norm().item() ** 2
                    grad_norm_sum += total_g ** 0.5
                    grad_n += 1

                optimizer.step()

                ep_loss_sum += loss.item() * x.size(0)
                ep_n += x.size(0)

        logs["epoch_loss"].append(ep_loss_sum / max(ep_n, 1))
        logs["grad_norm"].append(grad_norm_sum / max(grad_n, 1))

    # update norm (final minus initial)
    with torch.no_grad():
        total_u = 0.0
        for n, p in model.named_parameters():
            if n in init_params:
                total_u += (p.detach() - init_params[n]).float().norm().item() ** 2
        logs["update_norm"].append(total_u ** 0.5)

    return logs


In [ ]:
ranks = [8, 16]
subspaces = ["major", "minor", "random"]
seeds = [0, 1]   # reduce to [0] if still slow

def run_experiment(norm_cfg):
    rows = []

    print(f"\n===== Running BN variant: {norm_cfg} =====")

    for r in ranks:
        for kind in subspaces:
            for seed in seeds:
                set_seed(seed)

                model = load_baseline_model()
                model = apply_norm_config(model, norm_cfg)
                apply_subspace_constraints(model, conv_layers, kind, r, seed=seed)

                logs = train_with_logging(
                    model,
                    train_loaders,
                    epochs=3,
                    lr=1e-4
                )

                acc = eval_accuracy(model, test_loader)

                row = {
                    "norm_cfg": norm_cfg,
                    "rank": r,
                    "subspace": kind,
                    "seed": seed,
                    "target_acc": acc,
                    "gradnorm_mean": float(np.mean(logs["grad_norm"])),
                    "update_norm": logs["update_norm"][0],
                }
                rows.append(row)

                print(
                    f"[{norm_cfg}] r={r:2d} {kind:>6} seed={seed} "
                    f"acc={acc:.3f} gn={row['gradnorm_mean']:.3f} upd={row['update_norm']:.3f}"
                )

    return rows


In [ ]:
results_bn_default = run_experiment("bn_default")



===== Running BN variant: bn_default =====
[bn_default] r= 8  major seed=0 acc=0.734 gn=0.658 upd=4.545
[bn_default] r= 8  major seed=1 acc=0.734 gn=0.539 upd=4.621
[bn_default] r= 8  minor seed=0 acc=0.734 gn=0.611 upd=4.376
[bn_default] r= 8  minor seed=1 acc=0.722 gn=0.538 upd=4.545
[bn_default] r= 8 random seed=0 acc=0.735 gn=0.612 upd=4.453
[bn_default] r= 8 random seed=1 acc=0.727 gn=0.548 upd=4.718
[bn_default] r=16  major seed=0 acc=0.734 gn=0.664 upd=4.546
[bn_default] r=16  major seed=1 acc=0.723 gn=0.552 upd=4.625
[bn_default] r=16  minor seed=0 acc=0.733 gn=0.616 upd=4.397
[bn_default] r=16  minor seed=1 acc=0.722 gn=0.537 upd=4.584
[bn_default] r=16 random seed=0 acc=0.735 gn=0.616 upd=4.439
[bn_default] r=16 random seed=1 acc=0.728 gn=0.533 upd=4.628


In [ ]:
results_bn_affine = run_experiment("bn_affine_only")



===== Running BN variant: bn_affine_only =====
[bn_affine_only] r= 8  major seed=0 acc=0.734 gn=0.658 upd=4.545
[bn_affine_only] r= 8  major seed=1 acc=0.734 gn=0.539 upd=4.621
[bn_affine_only] r= 8  minor seed=0 acc=0.734 gn=0.611 upd=4.376
[bn_affine_only] r= 8  minor seed=1 acc=0.722 gn=0.538 upd=4.545
[bn_affine_only] r= 8 random seed=0 acc=0.735 gn=0.612 upd=4.453
[bn_affine_only] r= 8 random seed=1 acc=0.727 gn=0.548 upd=4.718
[bn_affine_only] r=16  major seed=0 acc=0.734 gn=0.664 upd=4.546
[bn_affine_only] r=16  major seed=1 acc=0.723 gn=0.552 upd=4.625
[bn_affine_only] r=16  minor seed=0 acc=0.733 gn=0.616 upd=4.397
[bn_affine_only] r=16  minor seed=1 acc=0.722 gn=0.537 upd=4.584
[bn_affine_only] r=16 random seed=0 acc=0.735 gn=0.616 upd=4.439
[bn_affine_only] r=16 random seed=1 acc=0.728 gn=0.533 upd=4.628


In [ ]:
results_bn_frozen = run_experiment("bn_frozen")



===== Running BN variant: bn_frozen =====
[bn_frozen] r= 8  major seed=0 acc=0.738 gn=0.609 upd=4.517
[bn_frozen] r= 8  major seed=1 acc=0.728 gn=0.503 upd=4.616
[bn_frozen] r= 8  minor seed=0 acc=0.735 gn=0.570 upd=4.373
[bn_frozen] r= 8  minor seed=1 acc=0.720 gn=0.501 upd=4.549
[bn_frozen] r= 8 random seed=0 acc=0.738 gn=0.574 upd=4.469
[bn_frozen] r= 8 random seed=1 acc=0.723 gn=0.509 upd=4.712
[bn_frozen] r=16  major seed=0 acc=0.732 gn=0.613 upd=4.511
[bn_frozen] r=16  major seed=1 acc=0.722 gn=0.517 upd=4.623
[bn_frozen] r=16  minor seed=0 acc=0.735 gn=0.577 upd=4.423
[bn_frozen] r=16  minor seed=1 acc=0.722 gn=0.500 upd=4.585
[bn_frozen] r=16 random seed=0 acc=0.739 gn=0.579 upd=4.466
[bn_frozen] r=16 random seed=1 acc=0.725 gn=0.501 upd=4.647


In [ ]:
results_groupnorm = run_experiment("groupnorm")



===== Running BN variant: groupnorm =====
[groupnorm] r= 8  major seed=0 acc=0.620 gn=8.665 upd=5.543
[groupnorm] r= 8  major seed=1 acc=0.583 gn=8.732 upd=5.609
[groupnorm] r= 8  minor seed=0 acc=0.605 gn=8.218 upd=5.565
[groupnorm] r= 8  minor seed=1 acc=0.557 gn=8.216 upd=5.664
[groupnorm] r= 8 random seed=0 acc=0.602 gn=8.246 upd=5.588
[groupnorm] r= 8 random seed=1 acc=0.567 gn=8.382 upd=5.552
[groupnorm] r=16  major seed=0 acc=0.600 gn=8.779 upd=5.480
[groupnorm] r=16  major seed=1 acc=0.576 gn=8.622 upd=5.563
[groupnorm] r=16  minor seed=0 acc=0.629 gn=8.261 upd=5.565
[groupnorm] r=16  minor seed=1 acc=0.562 gn=8.311 upd=5.603
[groupnorm] r=16 random seed=0 acc=0.643 gn=8.285 upd=5.556
[groupnorm] r=16 random seed=1 acc=0.588 gn=8.391 upd=5.633


In [ ]:
# =============================================================================
# Corrected Grad-Projection SoMA Subspace Swap (Fixes BN + Adam subtlety + freezing)
# =============================================================================
# Fixes vs your friend's original:
# 1) BatchNorm confound: BN running stats are frozen (BN in eval mode).
#    You can choose:
#      - bn_frozen      : freeze running stats AND freeze affine params
#      - bn_affine_only : freeze running stats, but train gamma/beta
# 2) Adam subtlety: use SGD with momentum=0 so projected gradients correspond
#    to projected updates (clean projected-gradient method).
# 3) Freezing outside convs: freeze all params by default, then selectively
#    unfreeze:
#      - the constrained conv weights (conv2 weights in selected layers)
#      - classifier head (fc)
#      - optional BN affine (if bn_affine_only)
#
# Assumptions:
# - PROJECT_ROOT, base_model_path exists
# - ResNet18_FeatureHook exposes .backbone like torchvision resnet
# - conv_layers is a dict {lname: (layer_name, block_idx, conv_name)}
# - make_balanced_loader / make_loader already defined
# - train_tf, test_tf, BATCH_SIZE already defined
# =============================================================================

import torch, numpy as np, random
import torch.nn as nn

def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ---------------------------
# Baseline loader
# ---------------------------
base_model_path = f"{PROJECT_ROOT}/resnet18_pacs_base.pt"

def load_baseline_model(path):
    model = ResNet18_FeatureHook(num_classes=7).to(device)
    state = torch.load(path, map_location=device)
    model.load_state_dict(state)
    return model

# ---------------------------
# Subspace basis
# ---------------------------
def get_soma_subspace(conv, kind, r, seed=0, layer_key=""):
    """
    Returns orthonormal basis U [C_out, r] for conv weight space (output-channel subspace).
    """
    W = conv.weight.detach().cpu().numpy()
    C_out = W.shape[0]
    Wmat = W.reshape(C_out, -1)
    U, S, _ = np.linalg.svd(Wmat, full_matrices=False)

    if kind == "major":
        return U[:, :r]
    if kind == "minor":
        return U[:, -r:]
    if kind == "middle":
        start = (C_out // 2) - (r // 2)
        return U[:, start:start + r]
    if kind == "random":
        # deterministic per-layer (optional); improves reproducibility across layers
        rng = np.random.default_rng(seed + (abs(hash(layer_key)) % 10_000))
        Q, _ = np.linalg.qr(rng.standard_normal((C_out, r)))
        return Q

    raise ValueError(f"Unknown subspace kind: {kind}")

# ---------------------------
# Gradient projection hook (with duplicate-hook guard)
# ---------------------------
def register_gradient_projection(param, U_np):
    """
    Projects grad onto span(U). Guard against duplicate hook stacking.
    """
    if getattr(param, "_has_soma_hook", False):
        return

    U = torch.tensor(U_np, dtype=torch.float32, device=param.device)
    U.requires_grad_(False)

    def hook(grad):
        # grad: [C_out, C_in, k, k]
        g = grad.view(grad.shape[0], -1)     # [C_out, D]
        g_proj = U @ (U.T @ g)               # [C_out, D]
        return g_proj.view_as(grad)

    h = param.register_hook(hook)
    param._has_soma_hook = True
    param._soma_hook_handle = h

def apply_subspace_constraints(model, conv_layers, kind, r, seed=0):
    """
    Applies gradient projection to the conv weight tensors of specified layers.
    """
    for lname, (layer, idx, conv_name) in conv_layers.items():
        conv = getattr(getattr(model.backbone, layer)[idx], conv_name)
        U = get_soma_subspace(conv, kind, r, seed=seed, layer_key=lname)
        register_gradient_projection(conv.weight, U)

# ---------------------------
# BN control
# ---------------------------
def set_bn_mode(model, mode: str):
    """
    mode in {"bn_default", "bn_frozen", "bn_affine_only"}
    """
    assert mode in {"bn_default", "bn_frozen", "bn_affine_only"}

    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            if mode == "bn_default":
                m.train()  # running stats update
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)
            elif mode == "bn_frozen":
                m.eval()   # freeze running stats
                m.weight.requires_grad_(False)
                m.bias.requires_grad_(False)
            elif mode == "bn_affine_only":
                m.eval()   # freeze running stats
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)

def enforce_bn_mode_during_training(model, mode: str):
    """
    model.train() will set BN modules to train mode unless we override.
    Call this after model.train(), and optionally inside the loop.
    """
    if mode == "bn_default":
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.train()
    else:
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

# ---------------------------
# Freeze / unfreeze policy (fixes "no freezing of layers outside convs")
# ---------------------------
def freeze_all_params(model):
    for p in model.parameters():
        p.requires_grad = False

def unfreeze_classifier(model):
    # torchvision resnet: backbone.fc
    for p in model.backbone.fc.parameters():
        p.requires_grad = True

def unfreeze_constrained_convs(model, conv_layers):
    """
    Only unfreeze the conv weights that are being projected.
    (Projection hook only matters if the parameter is trainable.)
    """
    for lname, (layer, idx, conv_name) in conv_layers.items():
        conv = getattr(getattr(model.backbone, layer)[idx], conv_name)
        conv.weight.requires_grad = True
        # conv bias in ResNet convs is typically None, but safe:
        if getattr(conv, "bias", None) is not None:
            conv.bias.requires_grad = True

# ---------------------------
# Train + Eval
# ---------------------------
@torch.no_grad()
def eval_accuracy(model, loader):
    model.eval()
    print("yes")
    correct, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(1)
        print("hmm")
        correct += (preds == y).sum().item()
        total += y.size(0)
        print("hmm")
    print("yeah")
    return correct / max(total, 1)

def train_with_constraints(model, train_loaders, bn_mode="bn_affine_only", epochs=3, lr=1e-4, weight_decay=0.0):
    """
    Uses SGD(m=0) to avoid Adam subtlety so projection is clean.
    Balanced loader list is preserved (one domain at a time).
    """
    # Train only params with requires_grad=True
    optimizer = torch.optim.SGD(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr, momentum=0.0, weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    for ep in range(epochs):
        model.train()
        enforce_bn_mode_during_training(model, bn_mode)

        for loader in train_loaders:
            for x, y in loader:
                x, y = x.to(device), y.to(device)

                enforce_bn_mode_during_training(model, bn_mode)

                optimizer.zero_grad(set_to_none=True)
                logits = model(x)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()


Device: cuda


In [ ]:
# =============================================================================
# RUN: Subspace Swap via Grad Projection (Corrected)
# =============================================================================

# ----- Data loaders (as in your friend's code; assumes these exist) -----
train_loaders = [
    make_balanced_loader("photo", train_tf, BATCH_SIZE),
    make_balanced_loader("art_painting", train_tf, BATCH_SIZE),
    make_balanced_loader("cartoon", train_tf, BATCH_SIZE),
]
test_loader = make_loader("sketch", test_tf, shuffle=False)

print("Tier-3 loaders ready (assumed).")

ranks = [4, 8, 16]
variants = ["minor", "major", "middle", "random"]

# Choose BN control for this experiment:
# - "bn_affine_only" is typically the best compromise for DG (stats frozen, affine trainable)
# - "bn_frozen" is stricter (affine frozen too)
BN_MODE = "bn_affine_only"

EPOCHS = 3
LR = 1e-4

tier3_results = {}

for r in ranks:
    print(f"\n=== Rank {r} ===")
    tier3_results[r] = {}
    for variant in variants:
        print(f"\nVariant: {variant}")

        for seed in [0]:  # add more seeds if you want
            set_seed(seed)

            # 1) Fresh baseline
            model = load_baseline_model(base_model_path)
            print("1")
            # 2) Freeze everything, then unfreeze only what we allow to adapt
            freeze_all_params(model)
            unfreeze_classifier(model)
            unfreeze_constrained_convs(model, conv_layers)
            print("2")
            # 3) Set BN mode (and ensure requires_grad matches)
            set_bn_mode(model, BN_MODE)
            print("3")

            # 4) Apply constraints (projection hooks) to the trainable conv weights
            apply_subspace_constraints(model, conv_layers, variant, r, seed=seed)
            print("4")

            # 5) Train
            train_with_constraints(model, train_loaders, bn_mode=BN_MODE, epochs=EPOCHS, lr=LR)
            print("5")

            # 6) Evaluate
            acc = eval_accuracy(model, test_loader)
            print("6")
            tier3_results[r].setdefault(variant, []).append(acc)

            print(f"  seed={seed} | Target accuracy: {acc:.3f}")

# Optional: summarize
print("\n==== Summary (mean over seeds) ====")
for r in ranks:
    for v in variants:
        vals = tier3_results[r].get(v, [])
        if vals:
            print(f"r={r:2d} {v:>6}: mean={np.mean(vals):.3f} std={np.std(vals):.3f} n={len(vals)}")


Tier-3 loaders ready (assumed).

=== Rank 4 ===

Variant: minor
1
2
3
4
5
yes
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hmm
hm

In [ ]:
import pandas as pd

df = pd.DataFrame(
    results_bn_default +
    results_bn_affine +
    results_bn_frozen +
    results_groupnorm
)

summary = (
    df.groupby(["norm_cfg", "rank", "subspace"])
      .agg(
          acc_mean=("target_acc", "mean"),
          acc_std=("target_acc", "std"),
          gradnorm=("gradnorm_mean", "mean"),
          update_norm=("update_norm", "mean"),
      )
      .reset_index()
      .sort_values(["norm_cfg", "rank", "acc_mean"], ascending=[True, True, False])
)

display(summary)

summary.to_csv(f"{PROJECT_ROOT}/tier3_bn_light_summary.csv", index=False)


,norm_cfg,rank,subspace,acc_mean,acc_std,gradnorm,update_norm
0,bn_affine_only,8,major,0.733902,0.000540,0.598962,4.582831
2,bn_affine_only,8,random,0.730720,0.005759,0.579620,4.585551
1,bn_affine_only,8,minor,0.727666,0.008639,0.574564,4.460609
5,bn_affine_only,16,random,0.731357,0.005219,0.574653,4.533898
3,bn_affine_only,16,major,0.728557,0.007379,0.608302,4.585496
4,bn_affine_only,16,minor,0.727793,0.007739,0.576844,4.490437
6,bn_default,8,major,0.733902,0.000540,0.598962,4.582831
8,bn_default,8,random,0.730720,0.005759,0.579620,4.585551
7,bn_default,8,minor,0.727666,0.008639,0.574564,4.460609
11,bn_default,16,random,0.731357,0.005219,0.574653,4.533898


# BatchNorm Confound Controls — Corrected + Stricter

In [ ]:
# =============================================================================
# BatchNorm Confound Controls — Corrected + Stricter
# =============================================================================
# What this experiment tests
# - Whether SoMA-style subspace-constrained learning effects are actually BN artifacts.
#
# What is fixed vs your draft:
#  (1) BN mode is NOT accidentally undone by model.train() during training.
#  (2) Gradient hooks are guarded to prevent accidental duplicate hook stacking.
#  (3) Optimizer caveat fixed: use SGD (momentum=0) so "projected gradient" = "projected update direction".
#  (4) Domain-imbalance caveat fixed: balanced per-step multi-domain training (equal contribution each step).
#
# Assumptions:
# - PACS is already exported to ImageFolder layout under PACS_ROOT.
# - You have a baseline source-trained model checkpoint.
# - Model exposes `model.backbone` like torchvision ResNet.
# - You define `PROJECT_ROOT`, `PACS_ROOT`, and `base_model_path`.
# =============================================================================

import os, json, random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from typing import Dict, Tuple, List

# ---------------------------
# Reproducibility
# ---------------------------
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ---------------------------
# Config
# ---------------------------
SOURCE_DOMAINS = ["photo", "art_painting", "cartoon"]
TARGET_DOMAIN = "sketch"

BATCH_SIZE = 32
NUM_WORKERS = 2
IMG_SIZE = 224

EPOCHS = 3
LR = 1e-4
WEIGHT_DECAY = 0.0

ranks = [8, 16]
subspaces = ["major", "minor", "random"]
seeds = [0, 1]

# Chosen layers where we constrain conv2 updates.
# conv_layers: dict {lname: (layer_name, block_idx, conv_name)}
conv_layers: Dict[str, Tuple[str, int, str]] = {
    "l2_b0_c2": ("layer2", 0, "conv2"),
    "l2_b1_c2": ("layer2", 1, "conv2"),
    "l3_b0_c2": ("layer3", 0, "conv2"),
    "l3_b1_c2": ("layer3", 1, "conv2"),
    "l4_b0_c2": ("layer4", 0, "conv2"),
    "l4_b1_c2": ("layer4", 1, "conv2"),
}

# ---------------------------
# Data
# ---------------------------
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

test_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def make_loader(domain: str, tfm, shuffle: bool):
    ds = datasets.ImageFolder(root=os.path.join(PACS_ROOT, domain), transform=tfm)
    return DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle,
                      num_workers=NUM_WORKERS, pin_memory=(device == "cuda"))

# Per-domain loaders so we can train with equal contribution per domain
train_loaders = [make_loader(d, train_tf, shuffle=True) for d in SOURCE_DOMAINS]
test_loader = make_loader(TARGET_DOMAIN, test_tf, shuffle=False)

# ---------------------------
# Baseline model loader
# ---------------------------
class ResNet18_Base(nn.Module):
    """Simple ResNet18 wrapper (adjust if your baseline class differs)."""
    def __init__(self, num_classes=7):
        super().__init__()
        self.backbone = models.resnet18(weights=None)
        self.backbone.fc = nn.Linear(512, num_classes)
    def forward(self, x):
        return self.backbone(x)

def load_baseline_model(path: str):
    m = ResNet18_Base(num_classes=7).to(device)
    state = torch.load(path, map_location=device)
    m.load_state_dict(state)
    return m

# ---------------------------
# Normalization controls
# ---------------------------
def set_bn_mode(model, mode: str):
    """
    mode in {"bn_default", "bn_frozen", "bn_affine_only"}
    """
    assert mode in {"bn_default", "bn_frozen", "bn_affine_only"}
    for m in model.modules():
        if isinstance(m, nn.BatchNorm2d):
            if mode == "bn_default":
                m.train()  # running stats update
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)
            elif mode == "bn_frozen":
                m.eval()   # freeze running stats
                m.weight.requires_grad_(False)
                m.bias.requires_grad_(False)
            elif mode == "bn_affine_only":
                m.eval()   # freeze running stats
                m.weight.requires_grad_(True)
                m.bias.requires_grad_(True)

def replace_bn_with_gn(module: nn.Module, num_groups: int = 32):
    """
    Recursively replace BatchNorm2d with GroupNorm.
    Copies affine params (gamma/beta) to reduce confound from random GN init.
    """
    for name, child in module.named_children():
        if isinstance(child, nn.BatchNorm2d):
            num_channels = child.num_features
            g = min(num_groups, num_channels)
            while num_channels % g != 0 and g > 1:
                g -= 1

            gn = nn.GroupNorm(num_groups=g, num_channels=num_channels, affine=True)

            # copy affine params if BN had affine
            with torch.no_grad():
                if child.affine:
                    gn.weight.copy_(child.weight.detach().cpu())
                    gn.bias.copy_(child.bias.detach().cpu())

            # move to same device
            gn = gn.to(next(module.parameters()).device if any(True for _ in module.parameters()) else device)
            setattr(module, name, gn)
        else:
            replace_bn_with_gn(child, num_groups=num_groups)

def apply_norm_config(model, norm_cfg: str):
    """
    norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}
    """
    assert norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}

    if norm_cfg == "groupnorm":
        replace_bn_with_gn(model, num_groups=32)
        # GN has no running stats; train() is fine
        return model
    else:
        set_bn_mode(model, norm_cfg)
        return model

def enforce_bn_behavior_during_training(model, norm_cfg: str):
    """
    Critical: model.train() will flip BN modules to train mode.
    We must re-enforce BN mode each time training starts and (optionally) each step.
    """
    if norm_cfg == "bn_default":
        # allow BN stats update
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.train()
        return

    if norm_cfg in {"bn_frozen", "bn_affine_only"}:
        # freeze running stats by keeping BN in eval mode
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

# ---------------------------
# Subspace basis + gradient projection
# ---------------------------
def get_soma_subspace(conv: nn.Conv2d, kind: str, r: int, seed: int = 0) -> np.ndarray:
    """
    Returns U basis [C_out, r] as numpy array (orthonormal).
    """
    assert kind in {"major", "minor", "middle", "random"}

    W = conv.weight.detach().cpu().numpy()
    C_out = W.shape[0]
    Wmat = W.reshape(C_out, -1)
    U, S, _ = np.linalg.svd(Wmat, full_matrices=False)

    if kind == "major":
        return U[:, :r]
    if kind == "minor":
        return U[:, -r:]
    if kind == "middle":
        start = (C_out // 2) - (r // 2)
        return U[:, start:start + r]
    if kind == "random":
        rng = np.random.default_rng(seed)
        Q, _ = np.linalg.qr(rng.standard_normal((C_out, r)))
        return Q

    raise ValueError(f"Unknown kind: {kind}")

def register_gradient_projection(param: torch.nn.Parameter, U_np: np.ndarray):
    """
    Project gradient onto span(U). U_np is numpy [C_out, r].
    Guard against duplicate hook stacking.
    """
    if getattr(param, "_has_soma_hook", False):
        return

    U = torch.tensor(U_np, dtype=torch.float32, device=param.device)
    U.requires_grad_(False)

    def hook(grad):
        # grad: [C_out, C_in, k, k]
        g = grad.view(grad.shape[0], -1)         # [C_out, D]
        g_proj = U @ (U.T @ g)                   # [C_out, D]
        return g_proj.view_as(grad)

    handle = param.register_hook(hook)
    param._has_soma_hook = True
    param._soma_hook_handle = handle  # in case you want to remove later

def apply_subspace_constraints(model, conv_layers: Dict[str, Tuple[str, int, str]],
                               kind: str, r: int, seed: int = 0):
    """
    Apply gradient projection to conv2 weights for selected layers.
    """
    for lname, (layer_name, idx, conv_name) in conv_layers.items():
        block = getattr(model.backbone, layer_name)[idx]
        conv = getattr(block, conv_name)
        U = get_soma_subspace(conv, kind, r, seed=seed)
        register_gradient_projection(conv.weight, U)

# ---------------------------
# Training / Evaluation
# ---------------------------
@torch.no_grad()
def eval_accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return correct / max(total, 1)

def train_with_logging(model, train_loaders: List[DataLoader], norm_cfg: str,
                       epochs: int = 3, lr: float = 1e-4, weight_decay: float = 0.0):
    """
    Balanced multi-domain training:
      - At each step, we take one batch from EACH domain loader
      - Accumulate gradients (averaged across domains)
      - Do one optimizer step
    This ensures equal contribution per domain, fixing the "domain imbalance" caveat.
    """
    # Use SGD with momentum=0 so projected grad corresponds to projected update direction.
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=lr,
        momentum=0.0,
        weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    logs = {"epoch_loss": [], "grad_norm": [], "update_norm": None}

    # snapshot params for update norm (trainable only)
    with torch.no_grad():
        init_params = {n: p.detach().clone() for n, p in model.named_parameters() if p.requires_grad}

    for ep in range(epochs):
        model.train()
        enforce_bn_behavior_during_training(model, norm_cfg)

        # Equalize steps across domains using the minimum loader length
        iters = [iter(ld) for ld in train_loaders]
        steps = min(len(ld) for ld in train_loaders)

        ep_loss_sum = 0.0
        ep_ex = 0
        grad_norm_sum = 0.0
        grad_n = 0

        for step in range(steps):
            optimizer.zero_grad()

            # accumulate gradients across domains, averaging losses
            loss_accum = 0.0
            for di, it_ in enumerate(iters):
                try:
                    x, y = next(it_)
                except StopIteration:
                    # should not happen given steps=min(len)
                    break
                x, y = x.to(device), y.to(device)

                # Keep BN behavior consistent even inside training loop
                enforce_bn_behavior_during_training(model, norm_cfg)

                logits = model(x)
                loss = criterion(logits, y) / len(train_loaders)
                loss.backward()
                loss_accum += loss.item() * x.size(0)  # scaled loss; ok for logging

                ep_ex += x.size(0)

            # grad norm after projection (hooks already applied)
            with torch.no_grad():
                total_g2 = 0.0
                for p in model.parameters():
                    if p.grad is not None:
                        total_g2 += float(p.grad.detach().float().norm().item() ** 2)
                grad_norm_sum += total_g2 ** 0.5
                grad_n += 1

            optimizer.step()
            ep_loss_sum += loss_accum

        logs["epoch_loss"].append(ep_loss_sum / max(ep_ex, 1))
        logs["grad_norm"].append(grad_norm_sum / max(grad_n, 1))

    # update norm (final minus initial)
    with torch.no_grad():
        total_u2 = 0.0
        for n, p in model.named_parameters():
            if n in init_params:
                total_u2 += float((p.detach() - init_params[n]).float().norm().item() ** 2)
        logs["update_norm"] = total_u2 ** 0.5

    return logs

# ---------------------------
# Experiment runner
# ---------------------------
def run_experiment(norm_cfg: str, base_model_path: str):
    """
    norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}
    """
    assert norm_cfg in {"bn_default", "bn_frozen", "bn_affine_only", "groupnorm"}
    rows = []
    print(f"\n===== Running Norm variant: {norm_cfg} =====")

    for r in ranks:
        for kind in subspaces:
            for seed in seeds:
                set_seed(seed)

                # fresh baseline each run
                model = load_baseline_model(base_model_path).to(device)

                # Apply normalization config
                model = apply_norm_config(model, norm_cfg).to(device)

                # Apply subspace constraints
                apply_subspace_constraints(model, conv_layers, kind, r, seed=seed)

                # Train + logs
                logs = train_with_logging(
                    model,
                    train_loaders=train_loaders,
                    norm_cfg=norm_cfg,
                    epochs=EPOCHS,
                    lr=LR,
                    weight_decay=WEIGHT_DECAY
                )

                acc = eval_accuracy(model, test_loader)

                row = {
                    "norm_cfg": norm_cfg,
                    "rank": r,
                    "subspace": kind,
                    "seed": seed,
                    "target_acc": float(acc),
                    "gradnorm_mean": float(np.mean(logs["grad_norm"])),
                    "update_norm": float(logs["update_norm"]),
                    "epoch_loss": [float(x) for x in logs["epoch_loss"]],
                    "epoch_grad_norm": [float(x) for x in logs["grad_norm"]],
                }
                rows.append(row)

                print(
                    f"[{norm_cfg}] r={r:2d} {kind:>6} seed={seed} "
                    f"acc={row['target_acc']:.3f} "
                    f"gn={row['gradnorm_mean']:.3f} upd={row['update_norm']:.3f}"
                )

    return rows

# ---------------------------
# Run all BN/GN configs and save
# ---------------------------
all_rows = []
for norm_cfg in ["bn_default", "bn_frozen", "bn_affine_only", "groupnorm"]:
    all_rows.extend(run_experiment(norm_cfg, base_model_path))

# Save to drive/project folder
os.makedirs(PROJECT_ROOT, exist_ok=True)
out_json = os.path.join(PROJECT_ROOT, "bn_confound_controls_results.json")
with open(out_json, "w") as f:
    json.dump(all_rows, f, indent=2)
print("\nSaved:", out_json)


Device: cuda

===== Running Norm variant: bn_default =====
[bn_default] r= 8  major seed=0 acc=0.716 gn=0.891 upd=0.003
[bn_default] r= 8  major seed=1 acc=0.721 gn=0.907 upd=0.003
[bn_default] r= 8  minor seed=0 acc=0.716 gn=0.874 upd=0.003
[bn_default] r= 8  minor seed=1 acc=0.721 gn=0.890 upd=0.003
[bn_default] r= 8 random seed=0 acc=0.716 gn=0.878 upd=0.003
[bn_default] r= 8 random seed=1 acc=0.721 gn=0.894 upd=0.003
[bn_default] r=16  major seed=0 acc=0.716 gn=0.905 upd=0.003
[bn_default] r=16  major seed=1 acc=0.721 gn=0.921 upd=0.003
[bn_default] r=16  minor seed=0 acc=0.716 gn=0.877 upd=0.003
[bn_default] r=16  minor seed=1 acc=0.721 gn=0.894 upd=0.003
[bn_default] r=16 random seed=0 acc=0.716 gn=0.884 upd=0.003
[bn_default] r=16 random seed=1 acc=0.721 gn=0.900 upd=0.003

===== Running Norm variant: bn_frozen =====
[bn_frozen] r= 8  major seed=0 acc=0.740 gn=4.191 upd=0.045
[bn_frozen] r= 8  major seed=1 acc=0.740 gn=4.011 upd=0.044
[bn_frozen] r= 8  minor seed=0 acc=0.740 gn=

In [ ]:
# ============================
# GROUPNORM ONLY (FAST/SAFE)
# ============================

# --- speed knobs ---
GN_EPOCHS = 2          # reduce from 3 if needed
GN_MAX_STEPS = 80     # cap steps per epoch (increase if you want more fidelity)
GN_SEEDS = [0]      # reduce to [0] if still slow
GN_RANKS = [8, 16]
GN_SUBSPACES = ["major", "minor", "random"]

def train_with_logging_groupnorm_fast(model, train_loaders, epochs=GN_EPOCHS, lr=1e-4, max_steps=GN_MAX_STEPS):
    # keep the same "projected-gradient => projected-update" purity
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.0, weight_decay=0.0)
    criterion = nn.CrossEntropyLoss()

    logs = {"epoch_loss": [], "grad_norm": [], "update_norm": []}

    # snapshot params for update norm
    with torch.no_grad():
        init_params = {n: p.detach().clone() for n, p in model.named_parameters() if p.requires_grad}

    for ep in range(epochs):
        model.train()  # GN has no running stats, so this is fine

        iters = [iter(ld) for ld in train_loaders]
        steps = min(len(ld) for ld in train_loaders)
        steps = min(steps, max_steps)  # <-- SPEED CAP

        ep_loss_sum, ep_ex = 0.0, 0
        grad_norm_sum, grad_n = 0.0, 0

        for _ in range(steps):
            optimizer.zero_grad()

            # balanced multi-domain accumulation (same as original)
            for it_ in iters:
                x, y = next(it_)
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = criterion(logits, y) / len(train_loaders)
                loss.backward()
                ep_loss_sum += loss.item() * x.size(0)
                ep_ex += x.size(0)

            # grad norm after projection hooks
            with torch.no_grad():
                total_g2 = 0.0
                for p in model.parameters():
                    if p.grad is not None:
                        total_g2 += float(p.grad.detach().float().norm().item() ** 2)
                grad_norm_sum += total_g2 ** 0.5
                grad_n += 1

            optimizer.step()

        logs["epoch_loss"].append(ep_loss_sum / max(ep_ex, 1))
        logs["grad_norm"].append(grad_norm_sum / max(grad_n, 1))

    # update norm
    with torch.no_grad():
        total_u2 = 0.0
        for n, p in model.named_parameters():
            if n in init_params:
                total_u2 += float((p.detach() - init_params[n]).float().norm().item() ** 2)
        logs["update_norm"].append(total_u2 ** 0.5)

    return logs


def run_groupnorm_only_fast(base_model_path):
    rows = []
    norm_cfg = "groupnorm"
    print(f"\n===== Running Norm variant: {norm_cfg} (FAST) =====")
    print(f"epochs={GN_EPOCHS}, max_steps/epoch={GN_MAX_STEPS}, seeds={GN_SEEDS}")

    for r in GN_RANKS:
        for kind in GN_SUBSPACES:
            for seed in GN_SEEDS:
                set_seed(seed)

                model = load_baseline_model(base_model_path)
                model = apply_norm_config(model, norm_cfg)

                apply_subspace_constraints(model, conv_layers, kind, r, seed=seed)

                logs = train_with_logging_groupnorm_fast(
                    model, train_loaders, epochs=GN_EPOCHS, lr=LR, max_steps=GN_MAX_STEPS
                )

                acc = eval_accuracy(model, test_loader)

                row = {
                    "norm_cfg": norm_cfg,
                    "rank": r,
                    "subspace": kind,
                    "seed": seed,
                    "target_acc": float(acc),
                    "gradnorm_mean": float(np.mean(logs["grad_norm"])),
                    "update_norm": float(logs["update_norm"][0]),
                    "epoch_loss": [float(x) for x in logs["epoch_loss"]],
                    "epoch_grad_norm": [float(x) for x in logs["grad_norm"]],
                    "max_steps": GN_MAX_STEPS,
                    "epochs": GN_EPOCHS,
                }
                rows.append(row)

                print(f"[{norm_cfg}] r={r:2d} {kind:>6} seed={seed} "
                      f"acc={acc:.3f} gn={row['gradnorm_mean']:.3f} upd={row['update_norm']:.3f}")

    return rows


# ---- RUN ONLY GROUPNORM (FAST) ----
gn_rows = run_groupnorm_only_fast(base_model_path)

# optional: save separately so you can merge later
import json, os
gn_out = os.path.join(PROJECT_ROOT, "bn_confound_controls_groupnorm_only_fast.json")
with open(gn_out, "w") as f:
    json.dump(gn_rows, f, indent=2)
print("\nSaved:", gn_out)



===== Running Norm variant: groupnorm (FAST) =====
epochs=2, max_steps/epoch=80, seeds=[0]


KeyboardInterrupt: 